In [36]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import re
from rapidfuzz import fuzz
import json

In [4]:
load_dotenv()

POSTGRES_HOST = "localhost"
POSTGRES_PORT = int(os.getenv("POSTGRES_PORT"))
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DB = os.getenv("POSTGRES_DB")

In [5]:
pg_conn = psycopg2.connect(
	host=POSTGRES_HOST,
	port=POSTGRES_PORT,
	user=POSTGRES_USER,
	password=POSTGRES_PASSWORD,
	dbname=POSTGRES_DB,
)

In [6]:
df = pd.read_sql("SELECT * FROM articles;", pg_conn)
df.shape

C:\Users\santi\AppData\Local\Temp\ipykernel_20240\1956988511.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM articles;", pg_conn)


(5626, 9)

In [7]:
df.head()

,id,title,description,content,url,source,published_at,collected_at,extra
0,0179bb3c8a09daf40c2bd43b83bcd356,Binance Founder CZ Criticizes an Altcoin Harsh...,,Binance founder and former CEO Changpeng Zhao ...,https://en.bitcoinsistemi.com/binance-founder-...,BitcoinSistemi,2025-10-03 16:07:08,2025-10-03 16:31:17.681133,"{'lang': 'EN', 'image': 'https://resources.cry..."
1,2c703f2f2d5855419f2a4f8cb6cb2575,Cardano Reserves On Coinbase Explode 462% Amid...,,The Cardano reserves on Coinbase have surged 4...,https://bitcoinist.com/coinbase-cardano-reserv...,Bitcoinist,2025-10-03 16:00:22,2025-10-03 16:31:17.681133,"{'lang': 'EN', 'image': 'https://resources.cry..."
2,87b146f36dd6baddd0182fd99da7bb35,UK May Allow Retail Trading of Bitcoin ETNs Ne...,,The UK will lift its retail ban on crypto exch...,https://en.coinotag.com/uk-may-allow-retail-tr...,CoinOtag,2025-10-03 15:58:01,2025-10-03 16:31:17.681133,"{'lang': 'EN', 'image': 'https://resources.cry..."
3,a78da56085595b097394d918c0a6ccd8,Bitcoin Price Prediction: $120K Uptober Rally ...,,"Bitcoin surged to $120,132 on Thursday, markin...",https://cryptonews.com/news/bitcoin-price-pred...,cryptonews,2025-10-02 23:58:08,2025-10-03 16:31:18.933505,"{'lang': 'EN', 'image': 'https://resources.cry..."
4,4d7c864c02d369982c5f3bd81d2f212b,Etsy Sellers Offer ‘Crypto Spells’ That Claim ...,,Etsy crypto spells are handcrafted listings wh...,https://en.coinotag.com/etsy-sellers-offer-cry...,CoinOtag,2025-10-02 23:16:18,2025-10-03 16:31:18.933505,"{'lang': 'EN', 'image': 'https://resources.cry..."


In [8]:
df.index = df.id
df.drop(columns=["id"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5626 entries, 0179bb3c8a09daf40c2bd43b83bcd356 to 63f7cd3de97041f92641a7f27839fb85
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         5626 non-null   object        
 1   description   5626 non-null   object        
 2   content       5626 non-null   object        
 3   url           5626 non-null   object        
 4   source        5626 non-null   object        
 5   published_at  5626 non-null   datetime64[ns]
 6   collected_at  5626 non-null   datetime64[ns]
 7   extra         5626 non-null   object        
dtypes: datetime64[ns](2), object(6)
memory usage: 395.6+ KB


In [9]:
df.drop(columns=["description"], inplace=True)

In [10]:
df.published_at = pd.to_datetime(df.published_at)
df.collected_at = pd.to_datetime(df.collected_at)

In [11]:
def clean_text(text):
    text = re.sub(r'[\n\t\r]', ' ', text)
    text = re.sub(r'[^\w\s.,;:!?\'"-]', '', text, flags=re.UNICODE)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = re.sub(r'https?://\S+', '<URL>', text)
    return text.strip()

df['title'] = df['title'].apply(clean_text)
df['content'] = df['content'].apply(clean_text)

# Despues ver nums, stopwords, etc.

In [12]:
(df['title'].str.len() >= df['content'].str.len()).sum()

np.int64(24)

In [13]:
df['title_longer_than_content'] = df['title'].str.len() >= df['content'].str.len()

In [14]:
df['title_num_words'] = df['title'].str.split().apply(len)
df['content_num_words'] = df['content'].str.split().apply(len)
df['title_num_chars'] = df['title'].str.len()
df['content_num_chars'] = df['content'].str.len()

for col in ['title_num_words', 'content_num_words', 'title_num_chars', 'content_num_chars']:
    print(f"{col}: {df[col].min()} - {df[col].max()}")

title_num_words: 3 - 28
content_num_words: 0 - 11518
title_num_chars: 16 - 162
content_num_chars: 0 - 62103


In [15]:
df.describe()

,published_at,collected_at,title_num_words,content_num_words,title_num_chars,content_num_chars
count,5626,5626,5626.000000,5626.000000,5626.000000,5626.000000
mean,2023-09-30 17:57:22.875577856,2025-10-03 16:47:12.099022592,11.538215,237.751333,73.405084,1494.299858
min,2021-08-26 21:30:06,2025-10-03 16:31:17.681133,3.000000,0.000000,16.000000,0.000000
25%,2022-08-04 21:55:49.750000128,2025-10-03 16:37:47.549765120,10.000000,30.000000,62.000000,210.000000
50%,2023-11-23 11:13:23.500000,2025-10-03 16:45:23.507728896,11.000000,56.000000,71.500000,363.500000
75%,2024-11-20 23:49:48.500000,2025-10-03 16:57:01.140826112,13.000000,383.750000,83.000000,2241.750000
max,2025-10-03 16:07:08,2025-10-03 17:03:45.560153,28.000000,11518.000000,162.000000,62103.000000
std,NaN,NaN,2.832079,404.349395,17.817514,2512.966811


In [16]:
df['title_to_content_len_ratio'] = df['title_num_chars'] / (df['content_num_chars'] + 1)
df['title_to_content_word_ratio'] = df['title_num_words'] / (df['content_num_words'] + 1)
token_pattern = re.compile(r'\b\w+\b')
df['content_tokens'] = df['content'].apply(lambda x: token_pattern.findall(x.lower()))

In [17]:
df['is_long_article'] = df['content_num_words'] > 450
df['is_short_article'] = df['content_num_words'] < 45
df['is_medium_article'] = (~df['is_long_article']) & (~df['is_short_article'])

In [18]:
btc_keywords = ["bitcoin", "bit coin" "btc", "satoshi", "cryptocurrency"]
df['is_btc_related'] = df[['title', 'content']].apply(
    lambda row: any(kw in row['title'] or kw in row['content'] for kw in btc_keywords),
    axis=1
)

df['btc_mention_count'] = df[['title', 'content']].apply(
    lambda row: sum(row['title'].count(kw) + row['content'].count(kw) for kw in btc_keywords),
    axis=1
)

In [19]:
df['is_duplicate_exact'] = df.duplicated(subset=['title', 'content'], keep=False)
df['is_duplicate_exact'].value_counts()
df = df[~df['is_duplicate_exact']]

In [20]:
df.btc_mention_count.value_counts(), df.is_btc_related.value_counts()

(btc_mention_count
 2     1227
 1     1010
 3      864
 0      578
 4      476
 7      373
 5      240
 6      163
 8       97
 9       61
 11      55
 13      46
 10      46
 12      44
 16      40
 15      38
 14      35
 18      33
 17      28
 19      25
 21      16
 20      16
 22      14
 23       8
 25       8
 39       5
 26       5
 28       5
 40       4
 24       4
 30       4
 33       4
 34       3
 29       3
 37       3
 41       3
 32       3
 27       3
 36       2
 42       2
 49       2
 38       2
 46       2
 31       2
 59       2
 55       2
 35       1
 50       1
 53       1
 51       1
 72       1
 47       1
 45       1
 78       1
 54       1
 56       1
 84       1
 62       1
 Name: count, dtype: int64,
 is_btc_related
 True     5040
 False     578
 Name: count, dtype: int64)

In [24]:
def fuzzy_duplicates(df, threshold=90):

    n = len(df)
    is_dup = np.zeros(n, dtype=bool)

    combined_texts = (
        df['title'].fillna('') + " " + df['content'].fillna('')
    ).tolist()

    for i in range(n):
        for j in range(i + 1, n):
            score = fuzz.token_set_ratio(combined_texts[i], combined_texts[j])
            if score >= threshold:
                is_dup[i] = True
                is_dup[j] = True
        if i % 500 == 0:
            print(f"[{i}/{n}] procesados...")

    return pd.Series(is_dup, index=df.index, name="is_duplicate_semantic")
df['is_duplicate_semantic'] = fuzzy_duplicates(df,90)
df['is_duplicate_semantic'].value_counts()

[0/5618] procesados...
[500/5618] procesados...
[1000/5618] procesados...
[1500/5618] procesados...
[2000/5618] procesados...
[2500/5618] procesados...
[3000/5618] procesados...
[3500/5618] procesados...
[4000/5618] procesados...
[4500/5618] procesados...
[5000/5618] procesados...
[5500/5618] procesados...


is_duplicate_semantic
False    5468
True      150
Name: count, dtype: int64

In [25]:
df = df[df['btc_mention_count'] >= 3]
min_content_words = 10
df = df[df['content_num_words'] >= min_content_words]
df = df[~df['is_duplicate_semantic']]

In [31]:
df = df.drop(columns=['is_btc_related', 'is_duplicate_exact', 'is_duplicate_semantic'], errors='ignore')
df.to_csv("../data/db/staged_articles.csv", index=True)

In [33]:
uri = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(uri)

In [37]:
for col in ['extra', 'content_tokens']:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (dict, list)) else x)

In [38]:
df.to_sql("production_layer", engine, index_label="id", if_exists="replace")

680